In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from collections import Counter

import functools

import math

import matplotlib.pyplot as plt

In [6]:
inputFile = "ID_data_mass_18122012_prepared.csv"

dataFrame = pd.read_csv(inputFile, header = 0, sep = ';')
print(dataFrame.shape)
data = dataFrame.values[0:10,0:3]
print(data)

numSamples = dataFrame.shape[0]
print("Num samples == " + str(numSamples))
numFeatures = dataFrame.shape[1]  #Last two features used as class designation
print("Num features == " + str(numFeatures))

kgfColumnId = dataFrame.shape[1] - 1
gtotalColumnId = dataFrame.shape[1] - 2

def classMarker(row):
    return str(line[kgfColumnId]) + '/' + str(line[gtotalColumnId])

(185, 33)
[[804 '05.06.08' 3576.3]
 [804 '06.06.08' 3576.3]
 [804 '07.06.08' 3576.3]
 [804 '08.06.08' 3576.3]
 [804 '09.06.08' 3576.3]
 [804 '17.06.11' 3576.3]
 [807 '16.09.04' 3590.0]
 [807 '16.09.04' 3590.0]
 [807 '17.09.04' 3590.0]
 [807 '17.09.04' 3590.0]]
Num samples == 185
Num features == 33


In [11]:
def giniCoefficient(groups, classes):
    numOfSamples = float(sum([len(group) for group in groups]))
    
    # Total Gini index
    result = 0.0
    
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
            
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
            
        groupWeight = (size / numOfSamples)
        result += (1.0 - score) * groupWeight
    return result

print(giniCoefficient([[[1, 1], [1, 0]], 
                       [[1, 1], [1, 0]]], 
                      [0, 1]))
print(giniCoefficient([[[1, 0], [1, 0]], 
                       [[1, 1], [1, 1]]], 
                      [0, 1]))

0.5
0.0


In [16]:
def splitData(featureIndex, splitValue, data):
    left = list()
    right = list()
    
    for row in data:
        if row[featureIndex] < splitValue:
            left.append(row)
        else:
            right.append(row)
            
    return left, right

print(splitData(0,
                1,
                [[0, 1], [2, 0]]))

([[0, 1]], [[2, 0]])
